In [1]:
# Import libraries
## for profiling
from profiler.main import profile

# for titiler-xarray
from typing import Any
import morecantile

# for titiler-pgstac
from titiler.pgstac.mosaic import PGSTACBackend
from typing import Any, Dict, List, Tuple
from rio_tiler.mosaic import mosaic_reader
from rio_tiler.models import ImageData
from geojson_pydantic import Polygon

# Only if wanting to see a picture
import io
from PIL import Image

In [2]:
import os
os.environ['AWS_PROFILE'] = 'veda-dev'

In [3]:
xyz_tile = (0,0,0)

## Profile titiler-pgstac

In [4]:
%load_ext autoreload
%autoreload
import pgstac.profile_pgstac as profile_pgstac 

### We can also run profiling via tile bench

In [5]:
#!pip install tilebench
#!tilebench profile s3://climatedashboard-data/cmip6/monthly/CMIP6_ensemble_median/tas/tas_month_ensemble-median_historical_195004.tif \
#    --tile 0-0-0 --add-cprofile --add-stdout --config GDAL_DISABLE_READDIR_ON_OPEN=EMPTY_DIR --config CPL_TIMESTAMP=ON | jq

In [6]:
query = {
  "collections": [
    "CMIP6_ensemble_median_TAS"
  ],
  "filter": {
    "op": "t_intersects",
    "args": [
      {
        "property": "datetime"
      },
      {
        "interval": [
          "1950-04-01T00:00:00Z",
          #"1950-05-01T00:00:00Z"
        ]
      }
    ]
  },
  "filter-lang": "cql2-json"
}

image_and_assets, logs = profile_pgstac.tile(*xyz_tile, query=dict(query))
logs

Tile exists time: 4.04
In rasterio WarpedVRT __ENTER__
time reading WarpedVRT: 571.6
time creating ImageData: 0.02
time for reader.part: 1001.74
{'pgstac-search': 71.41, 'get_tile': [[0.02, 3363.42]], 'mosaic': 3366.01}


{'cprofile': ['   ncalls  tottime  percall  cumtime  percall filename:lineno(function)',
  "       11    3.461    0.315    3.461    0.315 {method 'acquire' of '_thread.lock' objects}",
  '        4    0.084    0.021    0.086    0.021 connection.py:949(wait)',
  '        2    0.001    0.000    0.001    0.000 core.py:3433(__setmask__)',
  '        8    0.001    0.000    0.001    0.000 connection.py:434(_exec_command)']}

## Profile titiler-xarray

In [7]:
%load_ext autoreload
%autoreload
import xarray_tile_reader

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
image_and_assets, logs = xarray_tile_reader.tile(
    "cmip6-reference/combined_cmip6_kerchunk.json", 
    *xyz_tile,
    reference=True,
    variable="tas",
)

/Users/aimeebarciauskas/github/developmentseed/tile-benchmarking/profiling/zarr_reader.py:39: UserWarning: Variable(s) referenced in cell_measures not in variables: ['areacella']
  ds = xarray.open_dataset(src_path, **xr_open_args)


Time elapsed for xarray.open_datset: 796.16
Time elapsed for writing CRS: 4.64
Time elapsed for time slice: 0.84829
Time elapsed for dimension arrangement and array pre-processing: 19.35


TileOutsideBounds: Tile 0/0/0 is outside bounds

In [ ]:
content = image_and_assets.render(
    img_format='PNG'
)

im = Image.open(io.BytesIO(content))
im